In [3]:
import pandas as pd
import numpy as np
import random

np.random.seed(1)
random.seed(1)
TRAININFO = "/data/HitSongPrediction/train_info.tsv"
TRAINRANK =  "/data/HitSongPrediction/train_rank.csv"
TESTINFO = "/data/HitSongPrediction/test_info.tsv"
SUBMISSION = "/data/HitSongPrediction/submission.csv"

# Prepare data
df_i = pd.read_csv(TRAININFO, delimiter='\t',encoding='utf-8')
df_r = pd.read_csv(TRAINRANK)
df_i_train = df_i.merge(df_r, left_on='ID', right_on='ID')
df_i_train["dataset"] = "train"

df_i_test = pd.read_csv(TESTINFO, delimiter='\t',encoding='utf-8')
df_i_test["label"] = np.nan
df_i_test["dataset"] = "test"

df = pd.concat([df_i_train, df_i_test])
df_track_info = pd.read_csv("../../csv/all_track_info.csv")
df = df.merge(df_track_info, left_on='ID', right_on='ID')
df_audio_features = pd.read_csv("../../csv/all_track_audio_features.csv")
df =df.merge(df_audio_features,left_on="ID",right_on="ID", how="left")

# Sort by ID
df = df.sort_values(by=['ID'])
df= df.reset_index()

# for i,o in df.iterrows():
#     print(o.ID,"------",o.album,"--------", o.album_artist,"------",o.artist_name,"---",o.label)



In [4]:
import pandas as pd

pd.set_option('display.max_rows', 20)
df[["ID","title", "album", "album_artist","artist_name","artist_id","release_time","label"]]


,ID,title,album,album_artist,artist_name,artist_id,release_time,label
0,1073748245,Đêm Chôn Dầu Vượt Biển,30 Năm Viễn Xứ,Various Artists,Như Quỳnh,551,2017-10-01 22:07:00,7.0
1,1073751978,Mùa Thu Trong Mưa,Xin Còn Gọi Tên Nhau-Tình khúc Trường Sa,Various Artists,Minh Tuyết,455,2017-10-01 20:58:00,3.0
2,1073835561,Rồi Ánh Trăng Tan,Tình Mộng (Top Hits 29),Various Artists,Lưu Bích,450,2017-11-01 18:16:00,6.0
3,1073856553,Còn Thương Rau Đắng Mọc Sau Hè,Nối Lại Tình Xưa,Various Artists,Như Quỳnh,551,2017-11-01 17:36:00,2.0
4,1073929630,Người Điên Biết Yêu,Tình Lặng Câm,Như Loan,Như Loan,513,2017-11-01 17:49:00,7.0
...,...,...,...,...,...,...,...,...
10191,1078514050,Xin Còn Gọi Tên Nhau,The Wood 1,Phong Vũ,Phong Vũ,658098,2018-12-31 22:23:00,10.0
10192,1078514051,Delilah - Tình Hận,The Wood 1,Phong Vũ,Phong Vũ,658098,2018-12-31 22:23:00,9.0
10193,1078514052,Tình Yêu Như Mũi Tên,The Wood 1,Phong Vũ,Phong Vũ,658098,2018-12-31 22:23:00,9.0
10194,1078514053,Thu Hát Cho Người,The Wood 1,Phong Vũ,Phong Vũ,658098,2018-12-31 22:23:00,10.0


In [ ]:
df["album_right"] = df.groupby(df.release_time).ngroup().astype("category").cat.codes

df1=df[df.album_artist.isnull()]
for i,o in df1.iterrows():
    print(o.ID,"------",o.album_right,"--------", o.album_artist,"------",o.artist_name,"---",o.label)


In [5]:
pd.set_option('display.max_rows', 4000)

df22= df.groupby("album_right").agg({"label": ["count", "mean", "std"]})

df22[("label","std")] = df22[("label","std")].fillna(-2)
plt.hist(df22[("label","std")].values,bins=10)
plt.xlabel("Std of ranks of song in album")
plt.ylabel("Count")

KeyError: 'album_right'

In [ ]:
import re

def split_id(s):
    return re.split(',|\.',s)

m = df[df.dataset=="train"].artist_id.unique()
idx_lst = []
for idx in m:
    ps = split_id(idx)
    for i in ps:
        idx_lst.append(i)
        
train_artist = list(set(idx_lst))
train_album = df[df.dataset=="train"].album_right.unique()

def checkIsNewArtist(x):
    ids= split_id(x)
    for id in ids:
        if id in train_artist:
            return False
    return True
    
    
df_test = df[df.dataset=="test"]
df_test["isnewAlbum"] = df_test.album_right.apply(lambda x: x not in train_album)
df_test["isnewArtist"] = df_test.artist_id.apply(lambda x: checkIsNewArtist(x))
df_test["isnewArtistAndAlbum"] = df_test.isnewAlbum & df_test.isnewArtist





In [ ]:
df_test[df_test.isnewAlbum]

In [ ]:
df_test[df_test.isnewArtist]

In [ ]:
df_test[df_test.isnewArtistAndAlbum]

In [ ]:
from PIL import Image
import imagehash
import os
images_train_path = "/data/zalo/hit-song-prediction/train-image-samples"
images_test_path = "/data/zalo/hit-song-prediction/test-image-samples"

df["album_hash"] = df["album"] 
for i, row in df.iterrows():
    if row["dataset"]== "train":
        images_path = images_train_path
    else:
        images_path = images_test_path
    
    jpg_path = "/".join([images_path, str(row["ID"])+".mp3.jpg"])
    if os.path.isfile(jpg_path):
        df["album_hash"].iloc[i] = str(imagehash.average_hash(Image.open(jpg_path)))
        print(i)
    else:
        df["album_hash"].iloc[i] = np.nan

In [ ]:
pd.set_option('display.max_rows', 4000)
a = df[df.album_hash.isnull()]
a[["ID","title", "album", "album_artist","artist_name","artist_id","release_time","label"]]

In [ ]:
import re

def split_id(s):
    return re.split(',|\.',s)

m = df.artist_id.unique()
idx_lst = []
for idx in m:
    ps = split_id(idx)
    for i in ps:
        idx_lst.append(i)
        
id_lst = list(set(idx_lst))

def condition(df, artist_id):
    r = df.artist_id.apply(lambda x: artist_id in split_id(x))
    return r

df_train = df[df.dataset=="train"]
data= [df_train[condition(df_train, artist_id)].label.agg(["mean","std","count"]) for artist_id in id_lst]
new_df = pd.DataFrame(data=data)
new_df["artist_id"] =  id_lst



In [ ]:
import matplotlib.pyplot as plt

new_df["std"]= new_df["std"].fillna(-2)
plt.hist(new_df["std"], bins=10)
plt.xlabel("Std of ranks of songs perform by each artist")
plt.ylabel("Count")

In [ ]:
new_df["count"].value_counts().sort_index()

In [ ]:
df[df.artist_id=="729"]

In [ ]:
mean = new_df[new_df['count'] ==1]
mean

In [ ]:
mean_df = new_df[new_df['mean'] <3]
std_df = mean_df[mean_df['std']<2]
std_df['count'].value_counts().sort_index()